# Quantum Logic Testing

### General Setup

In [2]:
!pip install qiskit
!pip install pylatexenc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.7 MB 5.1 MB/s 
     |████████████████████████████████| 19.2 MB 1.2 MB/s 
     |████████████████████████████████| 240 kB 55.8 MB/s 
     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 112 kB 69.9 MB/s 
     |████████████████████████████████| 1.6 MB 62.7 MB/s 
     |████████████████████████████████| 943 kB 59.2 MB/s 
     |████████████████████████████████| 37.5 MB 21 kB/s 
     |████████████████████████████████| 49 kB 4.7 MB/s 
     |████████████████████████████████| 49 kB 4.7 MB/s 
     |████████████████████████████████| 4.0 MB 47.5 MB/s 
     |████████████████████████████████| 112 kB 63.1 MB/s 
  Created wheel for qiskit: filename=qiskit-0.38.0-py3-none-any.whl size=12147 sha256=7ec15fdf2a0e46c1ad9d96ffa1a468bac50fd802d16e134a62ad88cff7460ea6
  Stored in directory: /root/.cache/pip/wheels/0b/30/78/04c91b

In [3]:
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, AncillaRegister
from qiskit.visualization import plot_histogram
import numpy as np

### NOT Gate

In [4]:
def NOT(inp):
    """An NOT gate.
    
    Parameters:
        inp (str): Input, encoded in qubit 0.
        
    Returns:
        QuantumCircuit: Output NOT circuit.
        str: Output value measured from qubit 0.
    """

    qc = QuantumCircuit(1, 1) # A quantum circuit with a single qubit and a single classical bit
    qc.reset(0)
    
    # We encode '0' as the qubit state |0⟩, and '1' as |1⟩
    # Since the qubit is initially |0⟩, we don't need to do anything for an input of '0'
    # For an input of '1', we do an x to rotate the |0⟩ to |1⟩
    if inp=='1':
        qc.x(0)
        
    # barrier between input state and gate operation 
    qc.barrier()
    
    # Now we've encoded the input, we can do a NOT on it using x
    qc.x(0)
    
    #barrier between gate operation and measurement
    qc.barrier()
    
    # Finally, we extract the |0⟩/|1⟩ output of the qubit and encode it in the bit c[0]
    qc.measure(0,0)
    qc.draw('mpl')
    
    # We'll run the program on a simulator
    backend = Aer.get_backend('aer_simulator')
    # Since the output will be deterministic, we can use just a single shot to get it
    job = backend.run(qc, shots=1, memory=True)
    output = job.result().get_memory()[0]
    
    return qc, output

In [6]:
## Test the function
for inp in ['0', '1']:
    qc, out = NOT(inp)
    print('NOT with input',inp,'gives output',out)
    display(qc.draw())
    print('\n')

NOT with input 0 gives output 1


░ ┌───┐ ░ ┌─┐
  q: ─|0>──░─┤ X ├─░─┤M├
           ░ └───┘ ░ └╥┘
c: 1/═════════════════╩═
                      0



NOT with input 1 gives output 0


┌───┐ ░ ┌───┐ ░ ┌─┐
  q: ─|0>─┤ X ├─░─┤ X ├─░─┤M├
          └───┘ ░ └───┘ ░ └╥┘
c: 1/══════════════════════╩═
                           0

In [ ]:
def AND(inp1,inp2):
    """An AND gate.
    
    Parameters:
        inp1 (str): Input, encoded in qubit 0.
        inp2 (str): Input, encoded in qubit 1.
        
    Returns:
        QuantumCircuit: Output AND circuit.
        str: Output value measured from qubit 01.
    """
    qr = QuantumRegister(3, name = "q")
    ar = AncillaRegister(1, name = "ancilla")
    cr = ClassicalRegister(2, 'c')
    qc = QuantumCircuit(qr, ar, cr) # A quantum circuit with a 4 qubits and two classical bit
    qc.reset(range(3))
  
    if inp1=='1':
        qc.x(0)
    if inp2=='1':
        qc.x(1)
        
    qc.barrier()
    
    # Now we've encoded the input, we can do a reversible AND on it using ccx and cx
    # ccx does the AND and cx carrys the state of q0 to guarentee reversibility 
    qc.ccx(0,1,2)
    qc.cx(0,3)
    
    #barrier between gate operation and measurement
    qc.barrier()
    
    # Finally, we extract the |0⟩/|1⟩ output of the qubit and encode it in the bit c[0]
    qc.measure(2,1)
    qc.measure(3,0)
    qc.draw('mpl')
    
    # We'll run the program on a simulator
    backend = Aer.get_backend('aer_simulator')
    # Since the output will be deterministic, we can use just a single shot to get it
    job = backend.run(qc, shots=1, memory=True)
    output = job.result().get_memory()[0]
    
    return qc, output

In [ ]:
for inp1 in ['0', '1']:
    for inp2 in ['0', '1']:
        qc, output = AND(inp1, inp2)
        print('AND with inputs',inp1,inp2,'gives output',output)
        display(qc.draw())
        print('\n')

AND with inputs 0 0 gives output 00


░            ░       
    q_0: ─|0>──░───■────■───░───────
               ░   │    │   ░       
    q_1: ─|0>──░───■────┼───░───────
               ░ ┌─┴─┐  │   ░ ┌─┐   
    q_2: ─|0>──░─┤ X ├──┼───░─┤M├───
               ░ └───┘┌─┴─┐ ░ └╥┘┌─┐
ancilla: ──────░──────┤ X ├─░──╫─┤M├
               ░      └───┘ ░  ║ └╥┘
    c: 2/══════════════════════╩══╩═
                               1  0



AND with inputs 0 1 gives output 00


░            ░       
    q_0: ─|0>───────░───■────■───░───────
              ┌───┐ ░   │    │   ░       
    q_1: ─|0>─┤ X ├─░───■────┼───░───────
              └───┘ ░ ┌─┴─┐  │   ░ ┌─┐   
    q_2: ─|0>───────░─┤ X ├──┼───░─┤M├───
                    ░ └───┘┌─┴─┐ ░ └╥┘┌─┐
ancilla: ───────────░──────┤ X ├─░──╫─┤M├
                    ░      └───┘ ░  ║ └╥┘
    c: 2/═══════════════════════════╩══╩═
                                    1  0



AND with inputs 1 0 gives output 01


┌───┐ ░            ░       
    q_0: ─|0>─┤ X ├─░───■────■───░───────
              └───┘ ░   │    │   ░       
    q_1: ─|0>───────░───■────┼───░───────
                    ░ ┌─┴─┐  │   ░ ┌─┐   
    q_2: ─|0>───────░─┤ X ├──┼───░─┤M├───
                    ░ └───┘┌─┴─┐ ░ └╥┘┌─┐
ancilla: ───────────░──────┤ X ├─░──╫─┤M├
                    ░      └───┘ ░  ║ └╥┘
    c: 2/═══════════════════════════╩══╩═
                                    1  0



AND with inputs 1 1 gives output 11


┌───┐ ░            ░       
    q_0: ─|0>─┤ X ├─░───■────■───░───────
              ├───┤ ░   │    │   ░       
    q_1: ─|0>─┤ X ├─░───■────┼───░───────
              └───┘ ░ ┌─┴─┐  │   ░ ┌─┐   
    q_2: ─|0>───────░─┤ X ├──┼───░─┤M├───
                    ░ └───┘┌─┴─┐ ░ └╥┘┌─┐
ancilla: ───────────░──────┤ X ├─░──╫─┤M├
                    ░      └───┘ ░  ║ └╥┘
    c: 2/═══════════════════════════╩══╩═
                                    1  0

### ADDER logic

In [ ]:
def ADDER(inp1,inp2):
    """An ADDER
    
    Parameters:
        inpt1 (str): Input 1, encoded in qubit 0.
        inpt2 (str): Input 2, encoded in qubit 1.
        
    Returns:
        QuantumCircuit: Output ADDER circuit.
        str: Output value measured from qubit 2.
    """
    qr = QuantumRegister(3, 'q')
    ar = AncillaRegister(1, "ancilla")
    cr = ClassicalRegister(3, 'c')
    qc = QuantumCircuit(qr,ar,cr) 
    qc.reset(range(3))
  
    if inp1=='1':
        qc.x(0)
    if inp2=='1':
        qc.x(1)
        
    qc.barrier()

    # this is where your program for quantum AND gate goes

    qc.ccx(0, 1, 2) 
    qc.cx(0, 1) 
    qc.cx(0,3)
    qc.barrier()

    qc.measure(1, 0)  
    qc.measure(2, 1)   
    qc.measure(3, 2)    

    # We'll run the program on a simulator
    backend = Aer.get_backend('aer_simulator')
    # Since the output will be deterministic, we can use just a single shot to get it
    job = backend.run(qc,shots=1,memory=True)
    output = job.result().get_memory()
  
    return qc, output

In [ ]:
## Test the function
for inp1 in ['0', '1']:
    for inp2 in ['0', '1']:
        qc, output = ADDER(inp1, inp2)
        print('Adder with inputs',inp1,inp2,'gives output',output)
        display(qc.draw())
        print('\n')

Adder with inputs 0 0 gives output ['000']


░                 ░          
    q_0: ─|0>──░───■────■────■───░──────────
               ░   │  ┌─┴─┐  │   ░ ┌─┐      
    q_1: ─|0>──░───■──┤ X ├──┼───░─┤M├──────
               ░ ┌─┴─┐└───┘  │   ░ └╥┘┌─┐   
    q_2: ─|0>──░─┤ X ├───────┼───░──╫─┤M├───
               ░ └───┘     ┌─┴─┐ ░  ║ └╥┘┌─┐
ancilla: ──────░───────────┤ X ├─░──╫──╫─┤M├
               ░           └───┘ ░  ║  ║ └╥┘
    c: 3/═══════════════════════════╩══╩══╩═
                                    0  1  2



Adder with inputs 0 1 gives output ['001']


░                 ░          
    q_0: ─|0>───────░───■────■────■───░──────────
              ┌───┐ ░   │  ┌─┴─┐  │   ░ ┌─┐      
    q_1: ─|0>─┤ X ├─░───■──┤ X ├──┼───░─┤M├──────
              └───┘ ░ ┌─┴─┐└───┘  │   ░ └╥┘┌─┐   
    q_2: ─|0>───────░─┤ X ├───────┼───░──╫─┤M├───
                    ░ └───┘     ┌─┴─┐ ░  ║ └╥┘┌─┐
ancilla: ───────────░───────────┤ X ├─░──╫──╫─┤M├
                    ░           └───┘ ░  ║  ║ └╥┘
    c: 3/════════════════════════════════╩══╩══╩═
                                         0  1  2



Adder with inputs 1 0 gives output ['101']


┌───┐ ░                 ░          
    q_0: ─|0>─┤ X ├─░───■────■────■───░──────────
              └───┘ ░   │  ┌─┴─┐  │   ░ ┌─┐      
    q_1: ─|0>───────░───■──┤ X ├──┼───░─┤M├──────
                    ░ ┌─┴─┐└───┘  │   ░ └╥┘┌─┐   
    q_2: ─|0>───────░─┤ X ├───────┼───░──╫─┤M├───
                    ░ └───┘     ┌─┴─┐ ░  ║ └╥┘┌─┐
ancilla: ───────────░───────────┤ X ├─░──╫──╫─┤M├
                    ░           └───┘ ░  ║  ║ └╥┘
    c: 3/════════════════════════════════╩══╩══╩═
                                         0  1  2



Adder with inputs 1 1 gives output ['110']


┌───┐ ░                 ░          
    q_0: ─|0>─┤ X ├─░───■────■────■───░──────────
              ├───┤ ░   │  ┌─┴─┐  │   ░ ┌─┐      
    q_1: ─|0>─┤ X ├─░───■──┤ X ├──┼───░─┤M├──────
              └───┘ ░ ┌─┴─┐└───┘  │   ░ └╥┘┌─┐   
    q_2: ─|0>───────░─┤ X ├───────┼───░──╫─┤M├───
                    ░ └───┘     ┌─┴─┐ ░  ║ └╥┘┌─┐
ancilla: ───────────░───────────┤ X ├─░──╫──╫─┤M├
                    ░           └───┘ ░  ║  ║ └╥┘
    c: 3/════════════════════════════════╩══╩══╩═
                                         0  1  2

### Reversible Logic
Reversible are circuits (gates) that have one-to-one mapping between vectors of inputs and outputs;
thus the vector of input states can be always reconstructed from the vector of output states

### Feynman Gate

In [ ]:
def Feynman(inp1,inp2):
    """An Feynman
    
    Parameters:
        inpt1 (str): Input 1, encoded in qubit 0.
        inpt2 (str): Input 2, encoded in qubit 1.
        
    Returns:
        QuantumCircuit: Output Feynman circuit.
        str: Output value measured from qubit 2.
    """
    qc = QuantumCircuit(2, 1) 
   #qc.reset(range(2))
  
    if inp1=='1':
        qc.x(0)
    if inp2=='1':
        qc.x(1)
        
    qc.barrier()

    # this is where your program for quantum AND gate goes

    qc.cx(0, 1) 
    #qc.h( 1) 
    qc.barrier()
 
    qc.measure(1, 0)       

    # We'll run the program on a simulator
    backend = Aer.get_backend('aer_simulator')
    # Since the output will be deterministic, we can use just a single shot to get it
    job = backend.run(qc,shots=1,memory=True)
    output = job.result().get_memory()[0]
  
    return qc, output

In [ ]:
## Test the function
for inp1 in ['0', '1']:
    for inp2 in ['0', '1']:
        qc, output = Feynman(inp1, inp2)
        print('Feynman with inputs A (q_0) =',inp1, 'and B (q_1) =',inp2,'gives output Q =',output)
        display(qc.draw())
        print('\n')

Feynman with inputs A (q_0) = 0 and B (q_1) = 0 gives output Q = 0


░       ░    
q_0: ─░───■───░────
      ░ ┌─┴─┐ ░ ┌─┐
q_1: ─░─┤ X ├─░─┤M├
      ░ └───┘ ░ └╥┘
c: 1/════════════╩═
                 0



Feynman with inputs A (q_0) = 0 and B (q_1) = 1 gives output Q = 1


░       ░    
q_0: ──────░───■───░────
     ┌───┐ ░ ┌─┴─┐ ░ ┌─┐
q_1: ┤ X ├─░─┤ X ├─░─┤M├
     └───┘ ░ └───┘ ░ └╥┘
c: 1/═════════════════╩═
                      0



Feynman with inputs A (q_0) = 1 and B (q_1) = 0 gives output Q = 1


┌───┐ ░       ░    
q_0: ┤ X ├─░───■───░────
     └───┘ ░ ┌─┴─┐ ░ ┌─┐
q_1: ──────░─┤ X ├─░─┤M├
           ░ └───┘ ░ └╥┘
c: 1/═════════════════╩═
                      0



Feynman with inputs A (q_0) = 1 and B (q_1) = 1 gives output Q = 0


┌───┐ ░       ░    
q_0: ┤ X ├─░───■───░────
     ├───┤ ░ ┌─┴─┐ ░ ┌─┐
q_1: ┤ X ├─░─┤ X ├─░─┤M├
     └───┘ ░ └───┘ ░ └╥┘
c: 1/═════════════════╩═
                      0

### Fredkin Gate

In [ ]:
def Fredkin(inp1,inp2, inp3):
    """An Fredkin
    
    Parameters:
        inpt1 (str): Input 1, encoded in qubit 0.
        inpt2 (str): Input 2, encoded in qubit 1.
        inpt3 (str): Input 3, encoded in qubit 2.
        
    Returns:
        QuantumCircuit: Output Fredkin circuit.
        str: Output value measured .
    """
    qc = QuantumCircuit(3, 3) 
   #qc.reset(range(2))
  
    if inp1=='1':
        qc.x(0)
    if inp2=='1':
        qc.x(1)
    if inp3=='1':
        qc.x(2)
        
    qc.barrier()

    # this is where your program for quantum AND gate goes

    qc.cx(2, 1) 
    qc.cx(0, 1) 
    qc.h(2) 
    qc.t(0)
    qc.tdg(1)
    qc.t(2)
    qc.cx(2, 1) 
    qc.cx(0, 2) 
    qc.t(1)
    qc.cx(0, 1) 
    qc.tdg(2)
    qc.tdg(1)
    qc.cx(0, 2)
    qc.cx(2, 1)
    qc.t(1)
    qc.h(2)
    qc.cx(2, 1)
    qc.barrier()
 
    qc.measure(0, 2)
    qc.measure(1, 1)
    qc.measure(2, 0)       

    # We'll run the program on a simulator
    backend = Aer.get_backend('aer_simulator')
    # Since the output will be deterministic, we can use just a single shot to get it
    job = backend.run(qc,shots=1,memory=True)
    output = job.result().get_memory()
  
    return qc, output

In [ ]:
## Test the function
for inp1 in ['0', '1']:
    for inp2 in ['0', '1']:
      for inp3 in ['0', '1']:
        qc, output = Fredkin(inp1, inp2, inp3)
        print('Fredkin with inputs C (q_0) =',inp1, ', P (q_1) =',inp2,' and Q (q_2) = ', inp3, ' gives output PQR =',output)
        display(qc.draw())
        print('\n')

Fredkin with inputs C (q_0) = 0 , P (q_1) = 0  and Q (q_2) =  0  gives output PQR = ['000']


░            ┌───┐                                                   ░ »
q_0: ─░────────■───┤ T ├────────■───────────■───────────■──────────────────░─»
      ░ ┌───┐┌─┴─┐┌┴───┴┐┌───┐  │   ┌───┐ ┌─┴─┐┌─────┐  │  ┌───┐┌───┐┌───┐ ░ »
q_1: ─░─┤ X ├┤ X ├┤ Tdg ├┤ X ├──┼───┤ T ├─┤ X ├┤ Tdg ├──┼──┤ X ├┤ T ├┤ X ├─░─»
      ░ └─┬─┘├───┤└┬───┬┘└─┬─┘┌─┴─┐┌┴───┴┐└───┘└─────┘┌─┴─┐└─┬─┘├───┤└─┬─┘ ░ »
q_2: ─░───■──┤ H ├─┤ T ├───■──┤ X ├┤ Tdg ├────────────┤ X ├──■──┤ H ├──■───░─»
      ░      └───┘ └───┘      └───┘└─────┘            └───┘     └───┘      ░ »
c: 3/════════════════════════════════════════════════════════════════════════»
                                                                             »
«     ┌─┐      
«q_0: ┤M├──────
«     └╥┘┌─┐   
«q_1: ─╫─┤M├───
«      ║ └╥┘┌─┐
«q_2: ─╫──╫─┤M├
«      ║  ║ └╥┘
«c: 3/═╩══╩══╩═
«      2  1  0



Fredkin with inputs C (q_0) = 0 , P (q_1) = 0  and Q (q_2) =  1  gives output PQR = ['001']


░            ┌───┐                                                  »
q_0: ──────░────────■───┤ T ├────────■───────────■───────────■─────────────────»
           ░ ┌───┐┌─┴─┐┌┴───┴┐┌───┐  │   ┌───┐ ┌─┴─┐┌─────┐  │  ┌───┐┌───┐┌───┐»
q_1: ──────░─┤ X ├┤ X ├┤ Tdg ├┤ X ├──┼───┤ T ├─┤ X ├┤ Tdg ├──┼──┤ X ├┤ T ├┤ X ├»
     ┌───┐ ░ └─┬─┘├───┤└┬───┬┘└─┬─┘┌─┴─┐┌┴───┴┐└───┘└─────┘┌─┴─┐└─┬─┘├───┤└─┬─┘»
q_2: ┤ X ├─░───■──┤ H ├─┤ T ├───■──┤ X ├┤ Tdg ├────────────┤ X ├──■──┤ H ├──■──»
     └───┘ ░      └───┘ └───┘      └───┘└─────┘            └───┘     └───┘     »
c: 3/══════════════════════════════════════════════════════════════════════════»
                                                                               »
«      ░ ┌─┐      
«q_0: ─░─┤M├──────
«      ░ └╥┘┌─┐   
«q_1: ─░──╫─┤M├───
«      ░  ║ └╥┘┌─┐
«q_2: ─░──╫──╫─┤M├
«      ░  ║  ║ └╥┘
«c: 3/════╩══╩══╩═
«         2  1  0



Fredkin with inputs C (q_0) = 0 , P (q_1) = 1  and Q (q_2) =  0  gives output PQR = ['010']


...

┌───┐ ░            ┌───┐                                                  »
q_0: ┤ X ├─░────────■───┤ T ├────────■───────────■───────────■─────────────────»
     └───┘ ░ ┌───┐┌─┴─┐┌┴───┴┐┌───┐  │   ┌───┐ ┌─┴─┐┌─────┐  │  ┌───┐┌───┐┌───┐»
q_1: ──────░─┤ X ├┤ X ├┤ Tdg ├┤ X ├──┼───┤ T ├─┤ X ├┤ Tdg ├──┼──┤ X ├┤ T ├┤ X ├»
     ┌───┐ ░ └─┬─┘├───┤└┬───┬┘└─┬─┘┌─┴─┐┌┴───┴┐└───┘└─────┘┌─┴─┐└─┬─┘├───┤└─┬─┘»
q_2: ┤ X ├─░───■──┤ H ├─┤ T ├───■──┤ X ├┤ Tdg ├────────────┤ X ├──■──┤ H ├──■──»
     └───┘ ░      └───┘ └───┘      └───┘└─────┘            └───┘     └───┘     »
c: 3/══════════════════════════════════════════════════════════════════════════»
                                                                               »
«      ░ ┌─┐      
«q_0: ─░─┤M├──────
«      ░ └╥┘┌─┐   
«q_1: ─░──╫─┤M├───
«      ░  ║ └╥┘┌─┐
«q_2: ─░──╫──╫─┤M├
«      ░  ║  ║ └╥┘
«c: 3/════╩══╩══╩═
«         2  1  0



Fredkin with inputs C (q_0) = 1 , P (q_1) = 1  and Q (q_2) =  0  gives output PQR = ['101']


┌───┐ ░            ┌───┐                                                  »
q_0: ┤ X ├─░────────■───┤ T ├────────■───────────■───────────■─────────────────»
     ├───┤ ░ ┌───┐┌─┴─┐┌┴───┴┐┌───┐  │   ┌───┐ ┌─┴─┐┌─────┐  │  ┌───┐┌───┐┌───┐»
q_1: ┤ X ├─░─┤ X ├┤ X ├┤ Tdg ├┤ X ├──┼───┤ T ├─┤ X ├┤ Tdg ├──┼──┤ X ├┤ T ├┤ X ├»
     └───┘ ░ └─┬─┘├───┤└┬───┬┘└─┬─┘┌─┴─┐┌┴───┴┐└───┘└─────┘┌─┴─┐└─┬─┘├───┤└─┬─┘»
q_2: ──────░───■──┤ H ├─┤ T ├───■──┤ X ├┤ Tdg ├────────────┤ X ├──■──┤ H ├──■──»
           ░      └───┘ └───┘      └───┘└─────┘            └───┘     └───┘     »
c: 3/══════════════════════════════════════════════════════════════════════════»
                                                                               »
«      ░ ┌─┐      
«q_0: ─░─┤M├──────
«      ░ └╥┘┌─┐   
«q_1: ─░──╫─┤M├───
«      ░  ║ └╥┘┌─┐
«q_2: ─░──╫──╫─┤M├
«      ░  ║  ║ └╥┘
«c: 3/════╩══╩══╩═
«         2  1  0



Fredkin with inputs C (q_0) = 1 , P (q_1) = 1  and Q (q_2) =  1  gives output PQR = ['111']


┌───┐ ░            ┌───┐                                                  »
q_0: ┤ X ├─░────────■───┤ T ├────────■───────────■───────────■─────────────────»
     ├───┤ ░ ┌───┐┌─┴─┐┌┴───┴┐┌───┐  │   ┌───┐ ┌─┴─┐┌─────┐  │  ┌───┐┌───┐┌───┐»
q_1: ┤ X ├─░─┤ X ├┤ X ├┤ Tdg ├┤ X ├──┼───┤ T ├─┤ X ├┤ Tdg ├──┼──┤ X ├┤ T ├┤ X ├»
     ├───┤ ░ └─┬─┘├───┤└┬───┬┘└─┬─┘┌─┴─┐┌┴───┴┐└───┘└─────┘┌─┴─┐└─┬─┘├───┤└─┬─┘»
q_2: ┤ X ├─░───■──┤ H ├─┤ T ├───■──┤ X ├┤ Tdg ├────────────┤ X ├──■──┤ H ├──■──»
     └───┘ ░      └───┘ └───┘      └───┘└─────┘            └───┘     └───┘     »
c: 3/══════════════════════════════════════════════════════════════════════════»
                                                                               »
«      ░ ┌─┐      
«q_0: ─░─┤M├──────
«      ░ └╥┘┌─┐   
«q_1: ─░──╫─┤M├───
«      ░  ║ └╥┘┌─┐
«q_2: ─░──╫──╫─┤M├
«      ░  ║  ║ └╥┘
«c: 3/════╩══╩══╩═
«         2  1  0